In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

def find_Tc_from_diagram(rho_c, rho_d, temperatures, accuracy):
    
    '''Fit/extrapolate phase diagram and extract critical temperature and density'''
    
    def scaling_coex_densities(x, A, Tc_sc):
        beta = 0.325
        return A * (Tc_sc - x) ** beta

    def rectilinear_diametres(x, A2, Tc_rd, rho_cr): # (?) linear function should not have 3 parameters
        #ref : https://link.springer.com/article/10.1007/BF02847185
        return 2* rho_cr * (1 + A2* (Tc_rd- x)) # (?) -> ...
     
    popt, pcovt = curve_fit(scaling_coex_densities, temperatures, rho_c - rho_d, 
                            p0=[0.1, 400])

    popt2, pcovt2 = curve_fit(rectilinear_diametres, temperatures, (rho_c + rho_d), 
                                p0=[0.01, 400, 0.5])                            
    
    ext_T = np.linspace(np.min(temperatures), popt[1], accuracy)
    
    rho_drop_fit = (scaling_coex_densities(ext_T, *popt) + rectilinear_diametres(ext_T, *popt2)) / 2
    rho_solv_fit = (-scaling_coex_densities(ext_T, *popt) + rectilinear_diametres(ext_T, *popt2)) / 2
    
    fit = np.zeros(shape=(accuracy*2, 2))
    fit[:len(fit)//2, 0] = rho_solv_fit
    fit[len(fit)//2:, 0] = np.flip(rho_drop_fit)
    fit[:len(fit)//2, 1] = ext_T
    fit[len(fit)//2:, 1] = np.flip(ext_T)

    critical_T = popt[1] # popt2[1] is an incorrect parameter, too many parameters
    critical_rho= rho_solv_fit[-1] # popt2[1] is an incorrect parameter
    #cr_point = [critical_rho, critical_T[0]]

    return fit, critical_T, critical_rho

In [ ]:
#### parameters ####
accu= 500 # accuracy of fitting
clrC= ['#E01010', '#1010E0'] # color for plot
# -- constant
V= 14**2*(14*8) # volume in unit [nm^3]
C= 1000 # adjust the unit, can be changed

In [ ]:
#### load density profile ####
# Here the variable name is 'denCodC_pep, denDilC_pep, TempC', change the following code for loading
denCodC_pep=[]
denDilC_pep=[]
TempC=[]
NC= len(TempC)

In [ ]:
fitC_pep= []
TC_pep_cr= []
rhoC_pep_cr= []
for i in range(NC):
    denCod= denCodC_pep[i]
    denDil= denDilC_pep[i]
    Temp= TempC[i]
    if i_end==0:
        i_end= len(Temp)
    fit, T_cr, rho_cr= find_Tc_from_diagram(denCod, denDil, Temp, accu)
    fitC_pep.append(fit)
    TC_pep_cr.append(T_cr)
    rhoC_pep_cr.append(rho_cr)

In [ ]:
plt.rcParams.update({'font.size': 18})

plt.figure()
for i in range(NC):
    # seq= seqC[i]
    plt.plot(denDilC_pep[i]*C,TempC[i],marker='o',markersize=10, color=clrC[i],linestyle='none')
    plt.plot(denCodC_pep[i]*C,TempC[i],marker='o',markersize=10, color=clrC[i],linestyle='none')
    fit_pep= fitC_pep[i]
    plt.plot(fit_pep[:,0]*C,fit_pep[:,1],'--',linewidth=2, color=clrC[i])
    plt.scatter(rhoC_pep_cr[i]*C,TC_pep_cr[i], c=clrC[i],marker='o',s=100)

plt.legend(frameon=False, fontsize=14) # loc='lower|center|upper'
plt.xlabel(r'$\rho_{i}$ 10$^{-3}$ [nm$^{-3}$]')
plt.ylabel(r'$T$ [K]')
# plt.grid()
plt.tight_layout()
# plt.savefig('./phaseDiag_dens.png')